In [105]:
from matplotlib import pyplot as plt
import numpy as np
import ipywidgets as widgets
import pandas as pd
import matplotlib

Deposit

Mortgage

Slider to control size of each

Graph over time

Income is the key part and yield

Interest Rate Vary

In [135]:
def find_data(no_years, deposit_value, property_value, rental_income, 
              interest_rate, yearly_upkeep, extra_time = 0):
    mortgage_value = property_value - deposit_value
    array = np.arange(1, no_years+2+extra_time)
    income = np.ones(no_years+1+extra_time) * rental_income
    income[0] = 0
    
    
    mortgage_repayment = (mortgage_value * (1+interest_rate) ** no_years) / no_years
    costs = np.ones(no_years+1+extra_time) * (yearly_upkeep + mortgage_repayment)
    costs[0] = deposit_value
    
    repay = np.ones(no_years+1+extra_time) * mortgage_repayment
    repay[0] = 0
    
    
    df = pd.DataFrame(columns=['Year', 'Rental Income', 'Costs', 'Debt', 'Repay', 'Effective Debt', 'Sign', 'Bar_Top', 'Yearly', 'Yrsign'])
    df.Year = array
    df.Repay = repay
    df['Rental Income'] = income
    df['Costs'] = costs
    df.at[0, 'Debt'] = mortgage_value
    df.at[0, 'Effective Debt'] = mortgage_value + deposit_value
    repaid=False
    for i in range(len(df)-1):
        if not repaid:
            if ((1 + interest_rate) * df.loc[i, 'Debt']) - df.loc[i, 'Repay'] < 0:
                df.loc[i+1, 'Debt'] = 0
                df.loc[i, 'Repay'] = df.loc[i, 'Debt']
                df.loc[i, 'Costs'] = yearly_upkeep + df.loc[i, 'Repay']
                repaid = True
                
            else:
                df.loc[i+1, 'Debt'] = ((1 + interest_rate) * df.loc[i, 'Debt']) - df.loc[i, 'Repay']
                
            df.loc[i+1, 'Effective Debt'] = ((df.loc[i, 'Effective Debt'] + interest_rate * df.loc[i, 'Debt']) - df.loc[i, 'Repay'])
        else:
            df.loc[i+1, 'Debt'] = 0
            df.loc[i, 'Repay'] = 0
            df.loc[i, 'Costs'] = yearly_upkeep
            df.loc[i+1, 'Effective Debt'] = df.loc[i, 'Effective Debt'] - df.loc[i, 'Rental Income']
            
        if df.loc[i, 'Effective Debt'] > 0:
            df.loc[i, 'Sign'] = 'orange'
        else:
            df.loc[i, 'Sign'] = 'lime'
            
        df.loc[i, 'Bar_Top'] = abs(df.loc[i, 'Debt'] - df.loc[i, 'Effective Debt'])
        df.loc[i, 'Yearly'] = df.loc[i, 'Rental Income'] - df.loc[i, 'Costs']
        
        if df.loc[i, 'Yearly'] > 0:
            df.loc[i, 'Yrsign'] = 'lime'

        else:
            df.loc[i, 'Yrsign'] = 'red'
        
    if repaid:
        df.at[len(df)-1, 'Costs'] = yearly_upkeep
        df.at[len(df)-1, 'Repay'] = 0
        df.at[len(df)-1, 'Effective Debt'] = df.at[len(df)-2, 'Effective Debt'] - df.at[len(df)-1, 'Rental Income']
        
    if df.loc[len(df)-1, 'Effective Debt'] > 0:
        df.loc[len(df)-1, 'Sign'] = 'orange'
    else:
        df.loc[len(df)-1, 'Sign'] = 'lime'
    df.loc[len(df)-1, 'Yearly'] = df.loc[len(df)-1, 'Rental Income'] - df.loc[len(df)-1, 'Costs']
    if df.loc[len(df)-1, 'Yearly'] > 0:
        df.loc[len(df)-1, 'Yrsign'] = 'lime'
    else:
        df.loc[len(df)-1, 'Yrsign'] = 'red'
    df.loc[len(df)-1, 'Bar_Top'] = abs(df.loc[len(df)-1, 'Debt'] - df.loc[len(df)-1, 'Effective Debt'])
    df.loc[len(df)-1, 'Yearly'] = df.loc[len(df)-1, 'Rental Income'] - df.loc[len(df)-1, 'Costs']
    
    #print(df)
    
    
    
    # find yield
    total_costs = sum(df.Costs)
    property_yield = 100* rental_income / total_costs
    
    # plot pls
    
    plt.figure(figsize=[16, 9])
    gs = matplotlib.gridspec.GridSpec(2, 1, height_ratios=[4, 1], hspace=0)
    ax0=plt.subplot(gs[0])
    ax0.bar(df.Year, df.Debt, label='Mortgage')
    ax0.bar(df.Year, df.Bar_Top, bottom=df.Debt, color=df.Sign, label='Deposit')
    plt.ylabel('Cumulative Debt/Income')
    plt.legend()
    ax1=plt.subplot(gs[1])
    ax1.bar(df.Year, df.Yearly, color=df.Yrsign)
    plt.xlabel('Years')
    plt.ylabel('Yearly Income')
    plt.show()
    print('\n \n \n Property yield is: ' + str(property_yield))
    
#find_data(10, 30000, 150000, 16000, 0.02,1000, extra_time = 6)


## ADD FEATURES

# Negative income each year adds to 'deposit' bar top stuff
# Paying off mortgage leads to missing income not being accounted for properly in payoff year


In [137]:
no_years = widgets.IntSlider(value=10, min=5, max=50, step=1, description='Term Length', continuous_update=False)
property_value = widgets.IntSlider(value=150000, min=0, max=500000, step=1000, description='Property Val', continuous_update=False)
deposit_value = widgets.IntSlider(value=30000, min=0, max=150000, step=1000, description='Deposit', continuous_update=False)
rental_income = widgets.IntSlider(value=8000, min=0, max=30000, step=500, description='Rental Income', continuous_update=False)
interest_rate = widgets.FloatSlider(value=0.02, min=0, max=0.1, step=0.001, description='Interest Rate', continuous_update=False)
yearly_upkeep = widgets.IntSlider(value=1000, min=0, max=10000, step=250, description='Maintenance Cost', continuous_update=False)
extra_time = widgets.IntSlider(value=5, min=0, max=20, step=1, description='More Years', continuous_update=False)
lol = widgets.Label(description='Hello')

left = widgets.VBox([property_value, rental_income, yearly_upkeep, extra_time])
right = widgets.VBox([deposit_value, no_years, interest_rate, lol])

ui = widgets.HBox([left, right])

out = widgets.interactive_output(find_data, {'no_years':no_years, 'deposit_value':deposit_value, 'property_value':property_value,
                                             'rental_income':rental_income, 'interest_rate':interest_rate, 'yearly_upkeep':yearly_upkeep, 
                                             'extra_time':extra_time})

display(ui, out)

Output()

In [ ]:
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
div.input {
    display:none;
}
</style>
""")